In [48]:
import requests
from bs4 import BeautifulSoup
import re
#The basics
import numpy as np
import pandas as pd
import pickle

#Get them web sites
import requests

#Make sure slenium works
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#Start the google driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
#For inserting articles into Mongodb
from pymongo import MongoClient

In [49]:
NR_links = pickle.load(open("pickle_files/NR_links.pkl", "rb" ))
more_NR_links = pickle.load(open("pickle_files/more_NR_links.pkl", "rb" ))

In [50]:


def get_article(url):
#Sends request for url
    html = requests.get(url).text
    return html

def parse_article(html):
#BeautifulSoups the article
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('h1', class_='article-header__title').text
    body = soup.find('div', class_='article-content').text

    article = {
        'title': title,
        'body': body,
        'source': 'National Review',
        'num_source': 10
    }

    return article

def get_parsed_article_from_link(url):
#Runs the previous two functions on the url
    return parse_article(get_article(url))

In [51]:
#phase 1: uses request to try to BeautifulSoup links
NR_list_o_articles = []
NR_problem_articles = []
for text in NR_links:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        NR_list_o_articles.append(art)

    except:
        print("Problem processing url " + text)
        problem = text
        NR_problem_articles.append(problem)
    time.sleep(2)

Problem processing url https://www.nationalreview.com/photos/fuego-volcano-guatemala-eruptions/


In [61]:
#phase 2: uses selenium to go through links
driver = webdriver.Chrome(chromedriver)
for x in NR_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        soupy = BeautifulSoup(driver.page_source, 'lxml')
        title = soupy.find('h1', class_='article-header__title').text
        body = soupy.find('div', class_='article-content').text

        articley = {
        'title': title,
        'body': body,
        'source': 'National Review',
        'num_source': 10
        }
        NR_list_o_articles.append(articley)
    except:
        pass

In [53]:
#Same as above but for expanded list, phase 1
NR_more_list_o_articles = []
NR_more_problem_articles = []
for text in more_NR_links:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        NR_more_list_o_articles.append(art)

    except:
        print("Problem processing url " + text)
        problem = text
        NR_more_problem_articles.append(problem)
    time.sleep(2)

In [54]:
#same as above for expanded list, phase 2
driver = webdriver.Chrome(chromedriver)
for x in NR_more_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        soupy = BeautifulSoup(driver.page_source, 'lxml')
        title = soupy.find('h1', class_='article-header__title').text
        body = soupy.find('div', class_='article-content').text

        articley = {
        'title': title,
        'body': body,
        'source': 'National Review',
        'num_source': 10
        }
        NR_more_list_o_articles.append(articley)
    except:
        pass

In [62]:
all_articles = NR_list_o_articles + NR_more_list_o_articles

In [63]:
len(all_articles)

9

In [57]:
#starts client in Mongodb
client = MongoClient()
test = client.derp.test

In [58]:
#creates event and loads articles into Mongodb
db = client.events
test = db.test
test.insert_many(all_articles)


In [59]:
test.count()

67